In [20]:
import pandas as pd
import numpy
import seaborn as sns
import numpy as np
import networkx as nx
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt

In [21]:
df=pd.read_csv('dataset.csv')
df


,edge_id,length,minute,speed,is_holiday,weather
0,246,1726,505.0,22.140185,1.0,1.0
1,316,1750,740.0,54.303369,0.0,0.0
2,387,1612,745.0,56.303877,0.0,1.0
3,303,1927,1335.0,23.641228,0.0,2.0
4,250,1901,915.0,41.503161,0.0,1.0
...,...,...,...,...,...,...
496879,141,1842,440.0,17.536044,1.0,2.0
496880,226,1589,370.0,44.458878,0.0,0.0
496881,380,1862,245.0,47.966875,0.0,0.0
496882,138,1964,810.0,37.734834,0.0,0.0


In [22]:
df = df.sort_values(by = ['edge_id' , 'is_holiday' , 'weather' , 'minute'])
df

,edge_id,length,minute,speed,is_holiday,weather
37555,0,1618,0.0,65.332339,0.0,0.0
165409,0,1618,5.0,61.259386,0.0,0.0
391988,0,1618,10.0,59.867412,0.0,0.0
434141,0,1618,10.0,54.663338,0.0,0.0
18665,0,1618,15.0,65.527651,0.0,0.0
...,...,...,...,...,...,...
237907,389,1920,835.0,23.430464,1.0,2.0
124513,389,1920,1225.0,15.401880,1.0,2.0
59743,389,1920,1300.0,21.363083,1.0,2.0
210906,389,1920,1345.0,25.061533,1.0,2.0


In [23]:
df.fillna(method='ffill', inplace=True)

In [24]:
OL = df[ (df['speed'] < 10)  ].index
OL2 = df[ (df['speed'] > 80) ].index
df.drop(OL,inplace=True)
df.drop(OL2,inplace=True)

In [25]:
def duration(row): 
  return ( ( row['length'] /  (row['speed'] / 3.6 )  ) / 60 )
df['duration'] = df.apply(lambda row: duration(row), axis=1)

In [26]:
#  from sklearn.preprocessing import OneHotEncoder
#  ohe = OneHotEncoder()
#  transformed = ohe.fit_transform(df[['weather']])
#  df[ohe.categories_[0]] = transformed.toarray()

In [27]:
# df=df.drop(['weather'],axis=1)

In [28]:
def normalizer(column):
    maxx=column.max()
    minn=column.min()
    return (column-minn)/(maxx-minn)
df['length']=normalizer(df['length'])
# df['minute']=normalizer(df['minute'])
df['edge_id']=normalizer(df['edge_id'])

In [29]:
from sklearn.model_selection import train_test_split
X = pd.get_dummies(df.drop(['speed','duration','length'], axis=1))
y = df["duration"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12 , shuffle=True)
df

,edge_id,length,minute,speed,is_holiday,weather,duration
37555,0.0,0.808904,0.0,65.332339,0.0,0.0,1.485941
165409,0.0,0.808904,5.0,61.259386,0.0,0.0,1.584737
391988,0.0,0.808904,10.0,59.867412,0.0,0.0,1.621583
434141,0.0,0.808904,10.0,54.663338,0.0,0.0,1.775962
18665,0.0,0.808904,15.0,65.527651,0.0,0.0,1.481512
...,...,...,...,...,...,...,...
237907,1.0,0.959980,835.0,23.430464,1.0,2.0,4.916676
124513,1.0,0.959980,1225.0,15.401880,1.0,2.0,7.479607
59743,1.0,0.959980,1300.0,21.363083,1.0,2.0,5.392480
210906,1.0,0.959980,1345.0,25.061533,1.0,2.0,4.596686


In [30]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
model = RandomForestRegressor(n_estimators=200,max_depth=100,min_samples_split=20)
model.fit(X_train, y_train)
from sklearn.metrics import accuracy_score
predictions = model.predict(X_test)
print(model.score(X_test,y_test))


0.7632430866579198


In [31]:
model.score(X_train,y_train)

0.8239571302430483

In [32]:
#graph=nx.read_gpickle('network.gpickle')
!pip3 install pickle5
import pickle5 as pickle
with open('network.gpickle', "rb") as fh:
  g = pickle.load(fh)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
g.number_of_edges()

389

In [45]:
te=pd.read_csv('test_cases.csv')

In [46]:
def avali(start, end,minute,is_holiday,weather):
    test = pd.DataFrame()
    for i in range(390):
        test = test.append({'edge_id': i , 'minute': minute , 'is_holiday': is_holiday , 'weather': weather}, ignore_index=True)
    def normalizer(column):
        maxx=column.max()
        minn=column.min()
        return (column-minn)/(maxx-minn)
    test['edge_id']=normalizer(test['edge_id'])
    predictions = model.predict(test)
    for i in range(390):
      for j in range(390):
        if(g.has_edge(i,j)):
            g[i][j]['weight'] = predictions[g[i][j]['id']]
    path = nx.shortest_path(g,start,end)
    ans=0
    k = len(path)
    for i in range(k-1):
         ans+= g[path[i]][path[i+1]]['weight']
    ans
    return (ans)
def dovomi(start, end,minute,is_holiday,weather):
    test = pd.DataFrame()
    for i in range(390):
        test = test.append({'edge_id': i , 'minute': minute , 'is_holiday': is_holiday , 'weather': weather}, ignore_index=True)
    def normalizer(column):
        maxx=column.max()
        minn=column.min()
        return (column-minn)/(maxx-minn)
    test['edge_id']=normalizer(test['edge_id'])
    predictions = model.predict(test)
    for i in range(390):
      for j in range(390):
        if(g.has_edge(i,j)):
            g[i][j]['weight'] = predictions[g[i][j]['id']]
    path = nx.shortest_path(g,start,end)
    
    return (numpy.array(path))

In [47]:
eta = []
route = []
for i in range(10000):
    start=te.loc[i]['src']
    end=te.loc[i]['dest']
    minute=te.loc[i]['route_start_t']
    is_holiday=te.loc[i]['is_holiday']
    weather=te.loc[i]['weather']
    k = avali(start,end,minute,is_holiday,weather)
    eta.append(k*60)
    m = dovomi(start,end,minute,is_holiday,weather)
    route.append(m)

In [48]:
te['eta'] = eta
te['route'] = route
te

In [49]:
te

,src,dest,route_start_t,is_holiday,weather,eta,route
0,27,90,521,0,0,743.454986,"[27.0, 65.0, 143.0, 64.0, 36.0, 90.0]"
1,60,114,835,1,2,1354.902536,"[60.0, 86.0, 56.0, 163.0, 10.0, 114.0]"
2,18,112,668,0,0,776.517333,"[18.0, 36.0, 64.0, 107.0, 181.0, 112.0]"
3,96,18,1114,0,2,1020.220382,"[96.0, 57.0, 36.0, 18.0]"
4,66,135,1116,0,0,1153.662048,"[66.0, 62.0, 39.0, 83.0, 104.0, 135.0]"
...,...,...,...,...,...,...,...
9995,82,194,605,0,0,841.648318,"[82.0, 149.0, 4.0, 130.0, 100.0, 147.0, 194.0]"
9996,75,159,497,0,1,599.762539,"[75.0, 29.0, 22.0, 167.0, 159.0]"
9997,14,176,712,0,0,779.928402,"[14.0, 71.0, 85.0, 102.0, 50.0, 176.0]"
9998,14,77,1136,0,0,691.246574,"[14.0, 71.0, 85.0, 77.0]"
